# Part III: Flex Your Model!

In [ ]:
from pprint import pprint
import sys

import matplotlib
from matplotlib import pyplot
import numpy
import pandas
import pvlib

import pvfit
from pvfit.common import E_hemispherical_tilted_W_per_m2_stc, T_degC_stc
from pvfit.measurement.iv.types import (
    FTData,
    IVData,
    IVPerformanceMatrix,
)
from pvfit.modeling.dc.common import Material
import pvfit.modeling.dc.single_diode.equation.simple.simulation as sde_simple_sim
import pvfit.modeling.dc.single_diode.model.simple.auxiliary_equations as sdm_simple_ae
import pvfit.modeling.dc.single_diode.model.simple.inference_matrix as sdm_simple_inf_matrix
import pvfit.modeling.dc.single_diode.model.simple.inference_oc as sdm_simple_inf_oc

print("Python version: %s.%s.%s" % sys.version_info[:3])
print("matplotlib version: " + matplotlib.__version__)
print("numpy version: " + numpy.__version__)
print("pandas version: " + pandas.__version__)
print("pvlib version: " + pvlib.__version__)
print("pvfit version: " + pvfit.__version__)

## Configuration

### Module-Specific Parameters

In [ ]:
# Special thanks to PVPMC: https://pvpmc.sandia.gov/pv-research/pv-lifetime-project/pv-lifetime-modules/
# Special thanks to PV Performance Labs: https://datahub.duramat.org/en/dataset/gni-spectra-abq
# LG320N1K-A5 320W N-PERT Si, CFV Report #s 19074-PR-E-005 and 20125-PR-E-005
module_spec = {
    "make": "Canadian Solar",
    "model": "CS6K-275M",
    "length_mm": 1650,
    "width_mm": 992,
    "thickness_mm": 40,
    "material": Material.monoSi,
    "N_s": 60,
    "T_degC_0": T_degC_stc,
    "E_W_per_m2_0": E_hemispherical_tilted_W_per_m2_stc,
    # I-V Performance from IEC 61853-1.
    "I_sc_A_0": 9.299,
    "I_mp_A_0": 8.810,
    "P_mp_W_0": 277.37,
    "V_mp_V_0": 31.48,
    "V_oc_V_0": 38.29,
    "dI_sc_dT_A_per_degC_0": 0.00330,  # 0.0355 %/degC
    "dP_mp_dT_W_per_degC_0": -1.1522,  # -0.4155 %/degC
    "dV_oc_dT_V_per_degC_0": -0.1178,  # -0.3078 %/degC
    "I_sc_A": numpy.array([
        1.02432156592117, 2.04220329161805, 4.08876231489616, 6.16742104333028, 8.23620585618444, 10.3278357876755,
        1.02757453945704, 2.04102689488776, 4.0971414004165, 6.18719186830119, 8.26393082981699, 10.3551722873482, 11.395279503447,
        1.03913626907321, 2.06392255863692, 4.13658036822046, 6.23104469564041, 8.32545975924577, 10.4337969237837, 11.4824044494735,
        1.04491230559491, 2.0826858185294, 4.17166165651346, 6.27316031746301, 8.38355926709284, 10.4998465959987, 11.5723836702704,
    ]),
    "I_mp_A": numpy.array([
        0.97751989541629, 1.95726960371108, 3.92418023622383, 5.88128100651897, 7.83318220601609, 9.79347790957467,
        0.98916629640619, 1.94985466389299, 3.91453936347665, 5.86450863231455, 7.82315841412953, 9.78856875676654, 10.7347437361592,
        0.979284585841039, 1.9562110868987, 3.89343224765022, 5.85540132593549, 7.80002485702661, 9.7568852648073, 10.7216370906218,
        0.974720372318489, 1.952125140497, 3.88896895682736, 5.83030275932059, 7.76392690540913, 9.68589365355965, 10.6291077113922,
    ]),
    "V_mp_V": numpy.array([
        32.483559723954, 33.258802022921, 33.8340393593184, 34.0645574854619, 34.0867190147778, 33.9728268087745,
        30.9778660821221, 31.9811420769189, 32.594898159062, 32.8194215533058, 32.8550150165226, 32.7239339231629, 32.7049519389132,
        27.8308170148312, 28.7322982525907, 29.5538016200052, 29.6987353242727, 29.6990451824235, 29.5689893366802, 29.4925560748204,
        24.4491348564815, 25.448012412096, 26.219831977235, 26.427468974117, 26.4770877509191, 26.4207371130951, 26.4033443018523,
    ]),
    "V_oc_V": numpy.array([
        37.5801692934928, 38.6911107010062, 39.8079483178206, 40.4518064164751, 40.9107513118948, 41.2545791691551,
        36.3299943128803, 37.469502764561, 38.62911737847, 39.2968966959977, 39.7727741849426, 40.1323305920718, 40.2987316268181,
        33.1224743848133, 34.3345551259948, 35.5886134033995, 36.3181133114656, 36.8413940279752, 37.2287178272536, 37.4032160396215,
        29.7485796556728, 31.0865085511878, 32.4495485404176, 33.2484353792992, 33.8118471371682, 34.2675187516404, 34.4526444363561,
    ]),
    "E_W_per_m2": numpy.array([
        100, 200, 400, 600, 800, 1000,
        100, 200, 400, 600, 800, 1000, 1100,
        100, 200, 400, 600, 800, 1000, 1100,
        100, 200, 400, 600, 800, 1000, 1100,
    ]),
    "T_degC": numpy.array([
        15, 15, 15, 15, 15, 15,
        25, 25, 25, 25, 25, 25, 25,
        50, 50, 50, 50, 50, 50, 50,
        75, 75, 75, 75, 75, 75, 75,
    ]),
    "iam_angle_deg": numpy.array([0, 10, 20, 30, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90]),
    # IAM from IEC 61853-2.
    "iam_frac": numpy.array([
        1.0000, 0.9989, 1.0014, 1.0002, 0.9984, 0.9941, 0.9911, 0.9815, 0.9631, 0.9352,
        0.8922, 0.8134, 0.6778, 0.4541, 0.0000,
    ]),
    # DeltaT  measured in IEC 61853 standard.
    "sapm": {
        "DeltaT_degC": 3.0,  # Stock value.
    },
    # Temperature parameters from IEC 61853-2.
    "faiman": {
        "u0": 24.229,  # IEC 61853-2, actually installation dependent.
        "u1": 7.182,  # IEC 61853-2, actually installation dependent.
    },
    # Warranty degredation.
    "degredation": {
        "first_year": 0.025,
        "subsequent_years": 0.005,
    },
    # Spectral response.
    "sr": {
        "lambda_nm": numpy.array([
            300,  305,  310,  315,  320,  325,  330,  335,  340,  345,  350,
            355,  360,  365,  370,  375,  380,  385,  390,  395,  400,  405,
            410,  415,  420,  425,  430,  435,  440,  445,  450,  455,  460,
            465,  470,  475,  480,  485,  490,  495,  500,  505,  510,  515,
            520,  525,  530,  535,  540,  545,  550,  555,  560,  565,  570,
            575,  580,  585,  590,  595,  600,  605,  610,  615,  620,  625,
            630,  635,  640,  645,  650,  655,  660,  665,  670,  675,  680,
            685,  690,  695,  700,  705,  710,  715,  720,  725,  730,  735,
            740,  745,  750,  755,  760,  765,  770,  775,  780,  785,  790,
            795,  800,  805,  810,  815,  820,  825,  830,  835,  840,  845,
            850,  855,  860,  865,  870,  875,  880,  885,  890,  895,  900,
            905,  910,  915,  920,  925,  930,  935,  940,  945,  950,  955,
            960,  965,  970,  975,  980,  985,  990,  995, 1000, 1005, 1010,
        1015, 1020, 1025, 1030, 1035, 1040, 1045, 1050, 1055, 1060, 1065,
        1070, 1075, 1080, 1085, 1090, 1095, 1100, 1105, 1110, 1115, 1120,
        1125, 1130, 1135, 1140, 1145, 1150, 1155, 1160, 1165, 1170, 1175,
        1180, 1185, 1190, 1195, 1200
        ]),
        # Spectral response is actually normalized here.
        "S_A_per_W": numpy.array([
            0.        , 0.03730679, 0.07626343, 0.11847496, 0.16214371,
            0.20241547, 0.24121857, 0.27735138, 0.30130768, 0.31400299,
            0.3235321 , 0.33115387, 0.33811951, 0.34568787, 0.35356903,
            0.36102295, 0.36860657, 0.37688446, 0.38674927, 0.39775848,
            0.40844727, 0.41732788, 0.42365265, 0.42974091, 0.43779755,
            0.44664764, 0.45428467, 0.45993805, 0.46470642, 0.46937561,
            0.47473907, 0.4812851 , 0.48860168, 0.4961853 , 0.50349426,
            0.51046753, 0.51734924, 0.52415466, 0.53088379, 0.53749084,
            0.54402161, 0.55049133, 0.5569458 , 0.56338501, 0.56977844,
            0.57606506, 0.58227539, 0.58839417, 0.5944519 , 0.60043335,
            0.60636902, 0.61228943, 0.61819458, 0.62406921, 0.62991333,
            0.63572693, 0.64151001, 0.64724731, 0.65293884, 0.65858459,
            0.66418457, 0.66973877, 0.67523193, 0.68057251, 0.6857605 ,
            0.69091797, 0.69612122, 0.70152283, 0.70704651, 0.71263123,
            0.71836853, 0.72431946, 0.7305603 , 0.73770142, 0.74452209,
            0.74964905, 0.75404358, 0.75839233, 0.76341248, 0.76992798,
            0.77748108, 0.78486633, 0.7908783 , 0.79501343, 0.79846191,
            0.80264282, 0.80912781, 0.81687927, 0.82342529, 0.82795715,
            0.83171082, 0.83528137, 0.83921814, 0.84378052, 0.84854126,
            0.85316467, 0.85771179, 0.86224365, 0.86677551, 0.87138367,
            0.87609863, 0.88096619, 0.8860321 , 0.8914032 , 0.89706421,
            0.90284729, 0.90861511, 0.91445923, 0.9203949 , 0.92622375,
            0.9316864 , 0.93670654, 0.94146729, 0.9460907 , 0.95069885,
            0.95556641, 0.96055603, 0.96516418, 0.96893311, 0.97180176,
            0.97418213, 0.9763031 , 0.97834778, 0.98054504, 0.98309326,
            0.9866333 , 0.99104309, 0.99537659, 0.99868774, 1.        ,
            0.99981689, 0.99931335, 0.99848938, 0.99739075, 0.99601746,
            0.99200439, 0.98422241, 0.97460938, 0.96508789, 0.95632935,
            0.94744873, 0.93798828, 0.92750549, 0.91557312, 0.9017334 ,
            0.8842926 , 0.86245728, 0.83709717, 0.80908203, 0.77926636,
            0.74855042, 0.71430969, 0.67553711, 0.63542175, 0.59719849,
            0.5609436 , 0.52516174, 0.49068451, 0.45831299, 0.42822266,
            0.39994812, 0.3733902 , 0.34738159, 0.32099152, 0.29441071,
            0.26786041, 0.24153137, 0.21562958, 0.19035721, 0.16592407,
            0.14252472, 0.12036705, 0.09965515, 0.08058929, 0.06337738,
            0.04821968, 0.03532124, 0.02321959, 0.01180935, 0.00332475,
            0.
        ]),
    }
}

### Fit to simple SDM

In [ ]:
# Fit simple SDM to IEC-81853-1 data.
iv_performance_matrix = IVPerformanceMatrix(
    material=Material.monoSi,
    N_s=60,
    I_sc_A=module_spec["I_sc_A"],
    I_mp_A=module_spec["I_mp_A"],
    V_mp_V=module_spec["V_mp_V"],
    V_oc_V=module_spec["V_oc_V"],
    E_W_per_m2=module_spec["E_W_per_m2"],
    T_degC=module_spec["T_degC"],
    E_W_per_m2_0=E_hemispherical_tilted_W_per_m2_stc,
    T_degC_0=T_degC_stc,
)
sdm_simple_fit_matrix = sdm_simple_inf_matrix.fit(iv_performance_matrix=iv_performance_matrix)

# Examine fit parameters and fit quality.
pprint(sdm_simple_fit_matrix["model_parameters"])
pprint(sdm_simple_inf_matrix.compute_fit_quality(
    iv_performance_matrix=iv_performance_matrix,
    model_parameters=sdm_simple_fit_matrix["model_parameters"],
)[0])
sdm_simple_fit_matrix_iv_curve_parameters = sde_simple_sim.iv_curve_parameters(
    model_parameters=sdm_simple_ae.compute_sde_model_parameters(
        ft_data=FTData(F=iv_performance_matrix.F, T_degC=iv_performance_matrix.T_degC),
        model_parameters=sdm_simple_fit_matrix["model_parameters"],
    )
)
# Plot the data fits.
fig, ax = pyplot.subplots(figsize=(8, 6))
for idx, (F, T_degC) in enumerate(
    zip(iv_performance_matrix.F, iv_performance_matrix.T_degC)
):
    # Plot Isc, Pmp, and Voc with same colors as fit lines.
    color = next(ax._get_lines.prop_cycler)["color"]
    ax.plot(
        iv_performance_matrix.ivft_data.V_V[3 * idx : 3 * idx + 3],
        iv_performance_matrix.ivft_data.I_A[3 * idx : 3 * idx + 3],
        "o",
        color=color,
        mfc='none',
    )
    V_V_plot = numpy.linspace(0, iv_performance_matrix.ivft_data.V_V[3 * idx + 2], 101)
    ax.plot(
        V_V_plot,
        sde_simple_sim.I_at_V(
            V_V=V_V_plot,
            model_parameters=sdm_simple_ae.compute_sde_model_parameters(
                ft_data=FTData(F=F, T_degC=T_degC),
                model_parameters=sdm_simple_fit_matrix["model_parameters"],
            ),
        )["I_A"],
        label=f"F={F:.2f}, T={T_degC:.0f} °C",
        color=color,
    )
    ax.plot(0, sdm_simple_fit_matrix_iv_curve_parameters["I_sc_A"][idx], 'x', color=color)
    ax.plot(
        sdm_simple_fit_matrix_iv_curve_parameters["V_mp_V"][idx],
        sdm_simple_fit_matrix_iv_curve_parameters["I_mp_A"][idx],
        'x', color=color
    )
    ax.plot(sdm_simple_fit_matrix_iv_curve_parameters["V_oc_V"][idx], 0, 'x', color=color)

ax.set_title("6-Parameter Simple SDM Fit to Performance Matrix", fontdict={"fontsize": 14})
ax.set_xlabel("V (V)")
ax.set_ylabel("I (A)")

pyplot.show()

## Using a Reference PV Module as a Weather Station to Monitor (F, T)

### Synthetic Isc and and Voc measurements from the above calibrated reference PV module

In [ ]:
# FIXME Create pandas dataframe here for input and output.
timestamps = energy_timestamps = pandas.date_range(start="2014-05-09 05:15:00", end="2014-05-09 18:50:00", freq="5min", tz="Etc/GMT+7")
I_sc_A_meas = [0.03826451959460792, 0.09493217955802745, 0.1275366142280055, 0.16280377488648232, 0.16681518906280549, 0.19131243628346942, 0.21712220141566033, 0.2417222148262856, 0.2615230340858493, 0.28611680490153263, 0.31580034032435034, 0.38353350012008114, 0.48244523967360686, 0.6196900311005912, 0.7786317216761026, 0.970696245539592, 1.0842148297621672, 1.2260330782454316, 1.3932969651651865, 1.6500974937079658, 1.8881218333157912, 2.1219650857608383, 2.3457595444218695, 2.5354094998677414, 2.766218766447682, 2.992052390671814, 3.2019974645968046, 3.441004903947411, 3.678029785335956, 3.9251208949708944, 4.158227696865126, 4.4097642154931025, 4.786052432287301, 5.449652138088744, 1.8557123115876288, 1.6575937952355413, 1.2854269745625266, 1.5121767494010105, 2.807047008819191, 8.12152295752976, 2.4979478785439757, 1.2291114629143491, 1.6217790160796928, 2.6437316639050508, 5.501842332858465, 3.1743613284463015, 6.233737926926714, 9.293666147326862, 2.1694442437688175, 1.0378752896335406, 2.1463924940714927, 4.281799038406917, 8.84371677503106, 9.210044163371386, 9.201703983614262, 9.265736176187321, 9.9972893454278, 10.330614304019027, 5.50648303304816, 2.611665118841589, 2.544624186675098, 2.2894665948699466, 2.333571137990152, 2.850278978993103, 6.853550916741155, 3.912268774590667, 4.011363760571604, 6.274720469497806, 5.610784780381109, 11.684146967910968, 12.442787148577798, 7.123387223777935, 6.090367843486642, 5.214870869335765, 4.065705444255225, 2.6945488239383066, 2.1322043360712613, 2.2090685517217628, 2.3315859210310683, 1.2616474664676223, 1.4949493378818248, 4.927158568508131, 12.98706805465883, 12.007800390478579, 11.422043444868462, 11.840215514827861, 7.842886043895011, 11.847091157276408, 12.254705215811413, 2.8776289653866534, 14.03607146457023, 12.551681698782309, 12.230544898556643, 12.997332193244842, 11.716737023043095, 11.928719681982082, 11.783494329308306, 10.502147191339185, 2.7773383420631066, 4.427555471399343, 10.377189409870585, 10.33336272502082, 10.140152810910198, 10.065737652793738, 10.186755465969492, 9.993750365854021, 9.699643755425889, 10.565751084485278, 9.504835225579662, 9.311195944260819, 1.6188708105934602, 9.872658526969822, 6.921883204898357, 9.095030918144476, 8.667166552833867, 8.380686697791987, 4.536804314404423, 7.909630291421559, 7.650722207811073, 3.273859985116645, 3.6995130410716657, 7.510560258869946, 7.24872486867142, 6.764581727044709, 6.457662510408143, 6.214661571279557, 6.018887920623144, 5.798979656085738, 0.7800832545974843, 5.609583723600662, 5.113545281159606, 4.9736658495996195, 4.792950119225381, 3.637766235069103, 4.130560083619469, 3.8685969175902803, 2.0611219377985956, 2.4480591437707973, 3.0265040518296566, 2.3207319091424967, 2.868305936737152, 2.404245509253407, 2.247417860012913, 2.211035270988772, 1.9241698830054326, 1.892026897373587, 1.644912669227532, 1.3837208457324957, 1.3268995001641681, 1.0470056655849358, 0.8879829900548772, 0.8236826095030282, 0.7385768098343868, 0.6385685236495727, 0.5086936130828524, 0.363054755828676, 0.2525704749862826, 0.19655989420600545, 0.1416058728418215, 0.10624169379283678, 0.09418537617378324, 0.0589867297770845, 0.030376861354035844, 0.0003794795631635739]
V_oc_V_meas = [32.768644303364596, 34.557527148057126, 35.06966842827279, 35.468623045834924, 35.51061748733132, 35.712803531232126, 35.962378162378556, 36.22685673164613, 36.29043872105457, 36.35535085870981, 36.57381774340962, 36.79236573458544, 36.90937452599298, 37.30960995464313, 37.39197205193477, 37.590563318995756, 37.682726316195776, 37.77773306171372, 37.803013056513365, 37.98833910248711, 37.999637473198874, 38.6021583855205, 38.626233170106886, 38.33554356869133, 38.92105098190899, 38.95247892939201, 39.0199013347577, 39.12380280804608, 39.006060086009725, 39.30325600481425, 38.78357881906487, 39.25546713916555, 39.10479427117234, 39.48629978291188, 38.648593225267525, 38.57150198846001, 38.23104531433089, 38.393358509638944, 39.15375233984554, 39.295182829463066, 38.85021426005146, 38.09189799272812, 38.48444146073454, 39.12680943564586, 39.70634882165894, 39.47713135784585, 39.31319911342132, 38.7800304576247, 38.85570474494897, 37.91627908828334, 38.99729111186088, 39.61361325553813, 39.93937428667486, 39.69982423375917, 39.70650173747531, 39.66976709880863, 39.61598448231099, 39.30139860206904, 40.131364240027615, 39.482897427809206, 39.57488741959694, 39.49055393349848, 39.551423899317335, 39.815113222844, 40.498754838290985, 39.911043228412176, 40.01560204840677, 40.25391529069786, 39.83429636361605, 39.8650717824286, 39.54803133756771, 39.699998525088326, 40.05115599814965, 39.676306084451134, 39.54778051468261, 39.20918905310535, 39.0219595227007, 39.061731938650865, 39.11537451262906, 38.40308397931956, 38.567939320266895, 39.762480268842246, 39.53408167755694, 39.289555980943895, 39.48496141669022, 39.74731677507624, 39.84186633532986, 39.85138608187823, 39.68868370018265, 39.01260945875681, 38.410971232239774, 39.574552020968724, 40.036331936051326, 39.59726608468379, 39.627526413992896, 40.08134362613519, 40.07721081022274, 39.72215541475992, 38.803923743531215, 39.34268893472886, 40.094079875386, 39.57744224069771, 39.86869571344642, 39.89906709032844, 39.88126555439299, 39.72022817317061, 39.48924016821815, 39.94174096609507, 39.79901934223745, 39.440022161127956, 38.20117279221816, 38.92600121493284, 39.515239558760186, 39.82005519706772, 39.66975440518756, 39.556210100199486, 39.07297758030669, 39.39549939328688, 39.40962224291534, 38.97148360564175, 38.924772742451275, 39.449159497001666, 39.55534303079774, 39.369378951548235, 39.2524402441091, 39.446077658477954, 38.725282587711774, 39.03841803810515, 36.9387574827421, 39.032007917953635, 39.199935337423916, 38.99944844610469, 39.159097872422834, 38.80464611688478, 39.074544000540136, 38.8073817617095, 38.351119797832766, 38.58335887828042, 38.74684819167655, 38.49609207409268, 38.540814835861745, 38.51250913872361, 38.40241959324695, 38.355150049482845, 38.10752555176902, 38.24203332124046, 37.94113601998021, 37.82554141425589, 37.82741154229421, 37.54137571965448, 37.280320731583615, 37.2760508170176, 37.1264546620734, 36.91608178869875, 36.56352470548755, 36.09210995857521, 35.516252592278164, 35.1427287848087, 34.59124846703441, 34.0905918091825, 33.90769214661395, 33.05352942848537, 31.66067876008302, 1.137463746277179]

### Infer the (F, T) pair corresponding to each (Isc, Vov) pair in the time series.

In [ ]:
# FIXME Add output to pandas dataframe created above.
F = []
T_degC = []

for I_sc_A, V_oc_V in zip(I_sc_A_meas[:], V_oc_V_meas[:]):
    result = sdm_simple_inf_oc.fit(
        iv_data=IVData(V_V=numpy.array([0., V_oc_V]), I_A=numpy.array([I_sc_A, 0.])),
        model_parameters=sdm_simple_fit_matrix["model_parameters"],
    )

    F.append(result["oc_parameters"]["F"])
    T_degC.append(result["oc_parameters"]["T_degC"])

#### Summary Plots

In [ ]:
# FIXME Get data from pandas dataframe.
fig, ax = pyplot.subplots(nrows=4, ncols=1)

ax[0].plot(timestamps, I_sc_A_meas, '.', label="I_sc_A")
ax[0].set_ylabel("$I_\\mathrm{sc}$ [A]")
ax[0].legend(loc="upper left")

ax[1].plot(timestamps, V_oc_V_meas, '.', label="V_oc_V")
ax[1].set_ylabel("$V_\\mathrm{oc}$ [V]")
ax[1].legend(loc="lower left")

ax[2].plot(timestamps, F, '.', label="F")
ax[2].set_ylabel("$F$ [1]")
ax[2].legend(loc="upper left")

ax[3].plot(timestamps, T_degC, '.', label="T_degC")
ax[3].set_ylabel("T [$^\\circ$C]")
ax[3].legend(loc="upper left")

ax[0].set_title(
    '$(F, T)$ "weather" inferred from 5-min $(I_\\mathrm{sc}, V_\\mathrm{oc})$ module measurements'
)
fig.autofmt_xdate()
fig.tight_layout()